In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [30]:
# load dataframe
df = pd.read_parquet('amazon_user_reviews.parquet')

In [31]:
# understand df
df.head()

,category,rating,tokens,text_cleaned,original_text,token_count,title,images,verified_purchase,asin,parent_asin,user_id,datetime,helpful_vote
0,All_Beauty,5.0,"[great, party, lot, color, nice, give, cute, l...",great party lot color nice give cute look went...,These are great for parties. It has a lot of c...,25,great for parties,[],False,B07Z548TKH,B07Z548TKH,AGZZXSMMS4WRHHJRBUJZI4FZDHKQ,2020-08-12 21:05:33.567,1
1,All_Beauty,3.0,"[videoid, three, individually, wrapped, bath, ...",videoid three individually wrapped bath bomb s...,[[VIDEOID:f77804a8088c40dfbef6fdcfacf8b15a]] T...,132,cute mom gift but not as fragrant or impressiv...,"[{'attachment_type': 'IMAGE', 'large_image_url...",False,B095CG2ZV1,B095CG2ZV1,AFXF3EGQTQDXMRLDWFU7UBFQZB7Q,2021-09-11 13:31:15.148,1
2,All_Beauty,1.0,"[moisturizing, real, messy]",moisturizing real messy,Not very moisturizing and real messy.,3,one star,[],True,B00TP79D7A,B00TP79D7A,AFBSBG6VFGAHXSK4SZWLQDBKIFHQ,2015-09-13 01:35:02.000,0
3,All_Beauty,5.0,"[high, quality, tool, like, dentist, us, come,...",high quality tool like dentist us come storage...,"High quality tools, just like your dentist use...",13,"high quality tools,",[],False,B08QYW2FJ2,B08QYW2FJ2,AH7LDXK3GT67FZKMRW7MP477DJVQ,2021-04-20 04:21:07.963,0
4,All_Beauty,5.0,"[could, use, one, bundle, fantastic, hair, wou...",could use one bundle fantastic hair would litt...,You could use one bundle of this fantastic hai...,17,get two bundles for best results,[],False,B07Z7S97V8,B07Z7S97V8,AG73BVBKUOH22USSFJA5ZWL7AKXA,2020-04-21 02:16:00.515,0


In [ ]:
"""
product and categorical EDA:
visualize the difference in rating among products or categories
- Boxplot comparing categories
- Frequent product in categories
- Popularity of product in each categories
"""

'\nproduct and categorical EDA:\nvisualize the difference in rating among products or categories\n- Boxplot comparing categories\n- Frequent product in categories\n- Popularity of product in each categories\n\n'

In [ ]:
"""
time series EDA:
visualize the time and seasonal difference
- Yearly, monthly, day/night
"""

'\ntime series EDA:\nvisualize the time and seasonal difference\n- Yearly, monthly, day/night\n'